In [2]:
import pandas as pd
import numpy as np
import nltk

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [3]:
df = pd.read_csv("final_final_file.csv", sep='\t')

In [4]:
df.head()

,text,sentiment,bin_sentiment
0,something that i have been noticing recently i...,Catastrophizing,1
1,i went on vacation and just got back yesterday...,Catastrophizing,1
2,i've had crippling depression since i started ...,Jumping to conclusion,1
3,i feel like confusion is a huge part to depres...,Not distored,0
4,i skipped both of my classes yesterday. i had ...,Not distored,0


In [5]:
df_distorted_only = df[df['bin_sentiment'] == 1]

In [6]:
X, y = df_distorted_only['text'], df_distorted_only['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = 0.7, 
                                                    random_state = 42,
                                                    stratify = y)

# LOG + BOW

In [7]:
count_vectorizer = CountVectorizer(analyzer = "word", 
                                   tokenizer = nltk.word_tokenize,
                                   preprocessor = None, 
                                   stop_words = 'english', 
                                   min_df = 3)

train_data_features = count_vectorizer.fit_transform(X_train)

In [8]:
log_reg_BOW = LogisticRegression().fit(train_data_features, y_train)

BOW_log_features = count_vectorizer.transform(X_test)
y_pred = log_reg_BOW.predict(BOW_log_features)

C:\Users\stas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print(classification_report(y_test,y_pred, digits = 4))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.     0.2200    0.2157    0.2178        51
           Catastrophizing     0.2941    0.1724    0.2174        29
Disqualifying the positive     0.2593    0.2059    0.2295        34
       Emotional reasoning     0.2766    0.2407    0.2574        54
     Jumping to conclusion     0.2143    0.3158    0.2553        57
                  Labeling     0.2326    0.2857    0.2564        35
       Over-generalization     0.1538    0.1463    0.1500        41
           Personalization     0.2800    0.2258    0.2500        31
         Should statements     0.2800    0.2800    0.2800        25

                  accuracy                         0.2353       357
                 macro avg     0.2456    0.2320    0.2349       357
              weighted avg     0.2405    0.2353    0.2340       357

[0.21568627 0.17241379 0.20588235 0.24074074 0.31578947 0.28571429
 0.14634146 0.22580645 0.28      ]


# Grid: log + BOW

In [10]:
penalty = ['none', 'l1', 'l2', 'elasticnet']
C_first = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] # 1
C_second = [1, 2, 3, 4, 5, 6, 7, 8, 9]
solver = ['liblinear', 'saga', 'newton-cg', 'lbfgs']

param_grid = dict(penalty = penalty,
                  C = C_second,
                  solver = solver)

grid = GridSearchCV(estimator = log_reg_BOW,
                    param_grid = param_grid,
                    scoring = 'f1_weighted', #'recall'; ‘accuracy’; ‘precision’
                    verbose = 1,
                    n_jobs = -1)
grid_result = grid.fit(train_data_features, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 286 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 664 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  2.4min finished


In [11]:
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  0.2399485347355986
Best Params:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}


First iteration
Best Score:  0.2399485347355986
Best Params:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

Second iteration
Best Score:  0.2399485347355986
Best Params:  {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

In [12]:
log_reg_BOW = LogisticRegression(C = 1, penalty = "l2", solver = "liblinear").fit(train_data_features, y_train)

BOW_log_features = count_vectorizer.transform(X_test)
y_pred = log_reg_BOW.predict(BOW_log_features)

In [13]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

0.25770308123249297
                            precision    recall  f1-score   support

 Black and white thinking.       0.25      0.27      0.26        51
           Catastrophizing       0.29      0.17      0.22        29
Disqualifying the positive       0.28      0.21      0.24        34
       Emotional reasoning       0.31      0.28      0.29        54
     Jumping to conclusion       0.24      0.37      0.29        57
                  Labeling       0.24      0.26      0.25        35
       Over-generalization       0.16      0.15      0.15        41
           Personalization       0.27      0.23      0.25        31
         Should statements       0.33      0.32      0.33        25

                  accuracy                           0.26       357
                 macro avg       0.26      0.25      0.25       357
              weighted avg       0.26      0.26      0.26       357

[0.2745098  0.17241379 0.20588235 0.27777778 0.36842105 0.25714286
 0.14634146 0.22580645 0.3

# LOG + TF-IDF

In [14]:
tf_vect = TfidfVectorizer(min_df = 3,
                          tokenizer = nltk.word_tokenize,
                          preprocessor = None, 
                          stop_words = 'english')

TF_IDF_train_data_features = tf_vect.fit_transform(X_train)

logreg_TF_IDF = LogisticRegression().fit(TF_IDF_train_data_features, y_train)

In [15]:
data_features_TF_IDF = tf_vect.transform(X_test)
y_pred = logreg_TF_IDF.predict(data_features_TF_IDF)

# LOG + TF-IDF Grid search

In [16]:
penalty = ['none', 'l1', 'l2', 'elasticnet']
C_first = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] # 1
C_second = [10, 20, 30, 40, 50, 60, 70, 80, 90]
solver = ['liblinear', 'saga', 'newton-cg', 'lbfgs']

param_grid = dict(penalty = penalty,
                  C = C_second,
                  solver = solver)

grid = GridSearchCV(estimator = log_reg_BOW,
                    param_grid = param_grid,
                    scoring = 'f1_weighted', #'recall'; ‘accuracy’; ‘precision’
                    verbose = 1,
                    n_jobs = -1)
grid_result = grid.fit(TF_IDF_train_data_features, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  71 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:   30.4s


Best Score:  0.253568666162271
Best Params:  {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  1.2min finished


In [17]:
log_reg_BOW = LogisticRegression(C = 10, 
                                 penalty = "l2", 
                                 solver = "liblinear").fit(train_data_features, y_train)

BOW_log_features = count_vectorizer.transform(X_test)
y_pred = log_reg_BOW.predict(BOW_log_features)

In [18]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

0.22128851540616246
                            precision    recall  f1-score   support

 Black and white thinking.       0.19      0.22      0.20        51
           Catastrophizing       0.15      0.14      0.14        29
Disqualifying the positive       0.21      0.21      0.21        34
       Emotional reasoning       0.23      0.17      0.19        54
     Jumping to conclusion       0.25      0.32      0.28        57
                  Labeling       0.24      0.29      0.26        35
       Over-generalization       0.16      0.15      0.15        41
           Personalization       0.35      0.26      0.30        31
         Should statements       0.26      0.24      0.25        25

                  accuracy                           0.22       357
                 macro avg       0.23      0.22      0.22       357
              weighted avg       0.22      0.22      0.22       357

[0.21568627 0.13793103 0.20588235 0.16666667 0.31578947 0.28571429
 0.14634146 0.25806452 0.2

# LOG + doc2vec

In [26]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [27]:
train_data, test_data = train_test_split(df_distorted_only)

train_tagged = train_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), 
                             tags=[r['sentiment']]), axis = 1)

test_tagged = test_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), 
                             tags=[r['sentiment']]), axis = 1)

trainsent = train_tagged.values

doc2vec_model = Doc2Vec(trainsent, 
                        vector_size = 300, 
                        window = 5, #The maximum distance between the current and predicted word within a sentence. 
                        workers = 4,
                        epochs = 50,
                        dm = 1)

testsent = test_tagged.values
y_train, X_train = zip(*[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in trainsent])
y_test, X_test = zip(*[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in testsent])

In [28]:
logreg_doc2vec = LogisticRegression()
logreg_doc2vec.fit(X_train, y_train)
y_pred = logreg_doc2vec.predict(X_test)

In [29]:
print(classification_report(y_test,y_pred, digits = 4))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.     0.2195    0.2000    0.2093        45
           Catastrophizing     0.2381    0.2083    0.2222        24
Disqualifying the positive     0.1600    0.1333    0.1455        30
       Emotional reasoning     0.3158    0.3158    0.3158        38
     Jumping to conclusion     0.2000    0.2075    0.2037        53
                  Labeling     0.3077    0.4000    0.3478        30
       Over-generalization     0.0800    0.0606    0.0690        33
           Personalization     0.2143    0.2222    0.2182        27
         Should statements     0.4000    0.5882    0.4762        17

                  accuracy                         0.2391       297
                 macro avg     0.2373    0.2596    0.2453       297
              weighted avg     0.2271    0.2391    0.2310       297

[0.2        0.20833333 0.13333333 0.31578947 0.20754717 0.4
 0.06060606 0.22222222 0.58823529]


# LOG + doc2vec Grid search

In [30]:
penalty = ['none', 'l1', 'l2', 'elasticnet']
C_first = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000] # 1
C_second = [1, 2, 3, 4, 5, 6, 7, 8, 9]
C_third = [0.1, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
solver = ['liblinear', 'saga', 'newton-cg', 'lbfgs']

param_grid = dict(penalty = penalty,
                  C = C_third,
                  solver = solver)

grid = GridSearchCV(estimator = logreg_doc2vec,
                    param_grid = param_grid,
                    scoring = 'f1_weighted', #'f1_weighted'; ‘macro’; ‘precision’
                    verbose = 1,
                    n_jobs = -1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:   58.1s
[Parallel(n_jobs=-1)]: Done 450 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed:  3.5min finished


Best Score:  0.7534942492300717
Best Params:  {'C': 0.5, 'penalty': 'l2', 'solver': 'liblinear'}


In [31]:
logreg_doc2vec = LogisticRegression(C = 1, 
                                 penalty = "l2", 
                                 solver = "liblinear")
logreg_doc2vec.fit(X_train, y_train)
y_pred = logreg_doc2vec.predict(X_test)

In [32]:
print(classification_report(y_test,y_pred, digits = 4))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.     0.2174    0.2222    0.2198        45
           Catastrophizing     0.1429    0.0833    0.1053        24
Disqualifying the positive     0.1481    0.1333    0.1404        30
       Emotional reasoning     0.2683    0.2895    0.2785        38
     Jumping to conclusion     0.2241    0.2453    0.2342        53
                  Labeling     0.2647    0.3000    0.2812        30
       Over-generalization     0.1154    0.0909    0.1017        33
           Personalization     0.2500    0.2593    0.2545        27
         Should statements     0.3913    0.5294    0.4500        17

                  accuracy                         0.2290       297
                 macro avg     0.2247    0.2392    0.2295       297
              weighted avg     0.2185    0.2290    0.2220       297

[0.22222222 0.08333333 0.13333333 0.28947368 0.24528302 0.3
 0.09090909 0.25925926 0.52941176]


# SVM + TF-IDF

In [20]:
from sklearn import svm
from sklearn.svm import SVC

In [21]:
svc_tfidf = SVC().fit(TF_IDF_train_data_features, y_train)
data_features_TF_IDF = tf_vect.transform(X_test)
y_pred = svc_tfidf.predict(data_features_TF_IDF)

In [22]:
print(classification_report(y_test,y_pred))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.       0.23      0.31      0.26        51
           Catastrophizing       0.00      0.00      0.00        29
Disqualifying the positive       0.50      0.03      0.06        34
       Emotional reasoning       0.23      0.46      0.31        54
     Jumping to conclusion       0.20      0.56      0.30        57
                  Labeling       0.80      0.23      0.36        35
       Over-generalization       0.00      0.00      0.00        41
           Personalization       0.50      0.06      0.11        31
         Should statements       1.00      0.12      0.21        25

                  accuracy                           0.24       357
                 macro avg       0.38      0.20      0.18       357
              weighted avg       0.34      0.24      0.20       357

[0.31372549 0.         0.02941176 0.46296296 0.56140351 0.22857143
 0.         0.06451613 0.12      ]


C:\Users\stas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Grid:SVM + TF-IDF

In [23]:
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
C_second = [100, 200, 300, 400, 500, 600, 700, 800]
gamma = [0.0001, 0.001, 0.01, 0.00001, 0.000001, 0.00000001]
gamma_second = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
kernel = ['linear', 'poly', 'rbf', 'sigmoid'] #‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’

param_grid = dict(C = C,
                  gamma = gamma_second,
                  kernel = kernel)

grid = GridSearchCV(estimator = svc_tfidf,
                    param_grid = param_grid,
                    scoring = 'balanced_accuracy', #f1_weighted did not work
                    verbose = 1,
                    n_jobs = -1)
grid_result = grid.fit(TF_IDF_train_data_features, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.8min finished


Best Score:  0.2458842219967091
Best Params:  {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


- First iteration
Best Score:  0.2458842219967091
Best Params:  {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

- Second iteration (C_second + gamma)
Best Score:  0.2458842219967091
Best Params:  {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

- Third iteration (C_second + gamma_second)
Best Score:  0.24945291462095726 - best
Best Params:  {'C': 300, 'gamma': 0.006, 'kernel': 'sigmoid'}

- 4th iteration (C + gamma_second)
Best Score:  0.2458842219967091
Best Params:  {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [24]:
svc_tfidf = SVC(C = 300, 
                gamma = 0.006, kernel = 'sigmoid').fit(TF_IDF_train_data_features, y_train)
data_features_TF_IDF = tf_vect.transform(X_test)
y_pred = svc_tfidf.predict(data_features_TF_IDF)

In [25]:
print(classification_report(y_test,y_pred, digits = 6))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.   0.186667  0.274510  0.222222        51
           Catastrophizing   0.222222  0.137931  0.170213        29
Disqualifying the positive   0.227273  0.147059  0.178571        34
       Emotional reasoning   0.300000  0.444444  0.358209        54
     Jumping to conclusion   0.210526  0.280702  0.240602        57
                  Labeling   0.400000  0.285714  0.333333        35
       Over-generalization   0.129032  0.097561  0.111111        41
           Personalization   0.333333  0.193548  0.244898        31
         Should statements   0.250000  0.120000  0.162162        25

                  accuracy                       0.240896       357
                 macro avg   0.251006  0.220163  0.224591       357
              weighted avg   0.245841  0.240896  0.233240       357

[0.2745098  0.13793103 0.14705882 0.44444444 0.28070175 0.28571429
 0.09756098 0.19354839 0.12      ]


# SVM + doc2vec

In [33]:
SVM_doc2vec = SVC().fit(X_train, y_train)
y_pred = SVM_doc2vec.predict(X_test)

In [34]:
print(classification_report(y_test,y_pred, digits = 4))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.     0.2245    0.2444    0.2340        45
           Catastrophizing     0.2857    0.0833    0.1290        24
Disqualifying the positive     0.1765    0.1000    0.1277        30
       Emotional reasoning     0.3659    0.3947    0.3797        38
     Jumping to conclusion     0.2381    0.3774    0.2920        53
                  Labeling     0.2667    0.5333    0.3556        30
       Over-generalization     0.0000    0.0000    0.0000        33
           Personalization     0.2000    0.1111    0.1429        27
         Should statements     0.4286    0.1765    0.2500        17

                  accuracy                         0.2458       297
                 macro avg     0.2429    0.2245    0.2123       297
              weighted avg     0.2339    0.2458    0.2227       297

[0.24444444 0.08333333 0.1        0.39473684 0.37735849 0.53333333
 0.         0.11111111 0.17647059]


# Grid: SVM + doc2vec

In [35]:
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
C_second = [100, 200, 300, 400, 500, 600, 700, 800]
gamma = [0.0001, 0.001, 0.01, 0.00001, 0.000001, 0.00000001]
gamma_second = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

param_grid = dict(C = C,
                  gamma = gamma_second,
                  kernel = kernel)

grid = GridSearchCV(estimator = SVM_doc2vec,
                    param_grid = param_grid,
                    scoring = 'f1_weighted', #'f1_weighted'; ‘macro’; ‘precision’
                    verbose = 1,
                    n_jobs = -1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  2.6min finished


Best Score:  0.6910882404927561
Best Params:  {'C': 100, 'gamma': 0.003, 'kernel': 'rbf'}


- First iteration (C + gamma)
Best Score f1 weigted: 0.6909424496218526  
Best Params:  {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}

- Second iteration (C_second + gamma):
Best Score f1 weigted:  0.6946541766177363
Best Params:  {'C': 800, 'gamma': 0.001, 'kernel': 'rbf'}

- Third iteration (C_second + gamma_second)
Best Score:  0.6950169040140672 - BEST
Best Params:  {'C': 400, 'gamma': 0.002, 'kernel': 'rbf'}

- 4th iteration (C + gamma_second)
Best Score f1 weigted: 0.6942637273067429
Best Params:  {'C': 100, 'gamma': 0.006, 'kernel': 'rbf'}

In [36]:
SVM_doc2vec = SVC(C = 400, gamma = 0.002, kernel = "rbf").fit(X_train, y_train)
y_pred = SVM_doc2vec.predict(X_test)

In [37]:
print(classification_report(y_test,y_pred, digits = 4))

#print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

                            precision    recall  f1-score   support

 Black and white thinking.     0.2308    0.2000    0.2143        45
           Catastrophizing     0.1739    0.1667    0.1702        24
Disqualifying the positive     0.0938    0.1000    0.0968        30
       Emotional reasoning     0.2500    0.2895    0.2683        38
     Jumping to conclusion     0.2128    0.1887    0.2000        53
                  Labeling     0.2857    0.4000    0.3333        30
       Over-generalization     0.0455    0.0303    0.0364        33
           Personalization     0.3077    0.2963    0.3019        27
         Should statements     0.3636    0.4706    0.4103        17

                  accuracy                         0.2222       297
                 macro avg     0.2182    0.2380    0.2257       297
              weighted avg     0.2111    0.2222    0.2147       297

[0.2        0.16666667 0.1        0.28947368 0.18867925 0.4
 0.03030303 0.2962963  0.47058824]
